In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.applications import ResNet50

In [ ]:
base_dir = "/content/drive/My Drive/chromosome/"
img_width = 128
img_height = 128
batch_size = 2
os.listdir(base_dir)

['3',
 '24',
 '23',
 '8',
 '7',
 '5',
 '4',
 '9',
 '6',
 '22',
 '13',
 '21',
 '19',
 '15',
 '16',
 '17',
 '2',
 '20',
 '14',
 '18',
 '11',
 '10',
 '12',
 '1']

In [ ]:
datagen = ImageDataGenerator(
    rescale =1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    validation_split = 0.3)

In [ ]:
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (img_width,img_height),
    batch_size = batch_size,
    subset="training",
    class_mode="categorical")

Found 3842 images belonging to 24 classes.


In [ ]:
val_datagen = ImageDataGenerator(
    rescale =1. / 255)

In [ ]:
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (img_width,img_height),
    batch_size = batch_size,
    subset="validation",
    class_mode="categorical")

Found 1632 images belonging to 24 classes.


In [ ]:
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(128,128,3))
for layer in restnet.layers:
  layer.trainable = False
output = restnet.layers[-1].output
output = tf.keras.layers.Flatten()(output)
output = tf.keras.layers.Dense(512, activation='relu')(output)
output = tf.keras.layers.Dropout(0.25)(output)
output = tf.keras.layers.Dense(24, activation='softmax')(output)
restnet = tf.keras.models.Model(restnet.input, outputs=output)



restnet.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
restnet.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])

In [ ]:
restnet.fit_generator(
    train_generator,
    steps_per_epoch = 1921,
    epochs = 50,
    validation_data=val_generator,
    validation_steps = 816)

Epoch 1/100
206/206 [==============================] - 27s 130ms/step - loss: 2.7898 - accuracy: 0.1966 - val_loss: 9.1251 - val_accuracy: 0.0482
Epoch 2/100
206/206 [==============================] - 26s 125ms/step - loss: 2.3939 - accuracy: 0.2519 - val_loss: 6.5845 - val_accuracy: 0.0394
Epoch 3/100
206/206 [==============================] - 26s 125ms/step - loss: 1.8232 - accuracy: 0.4183 - val_loss: 6.0864 - val_accuracy: 0.0394
Epoch 4/100
206/206 [==============================] - 26s 125ms/step - loss: 1.6667 - accuracy: 0.4736 - val_loss: 6.7695 - val_accuracy: 0.0570
Epoch 5/100
206/206 [==============================] - 26s 125ms/step - loss: 1.6533 - accuracy: 0.4758 - val_loss: 2.9347 - val_accuracy: 0.1388
Epoch 6/100
206/206 [==============================] - 26s 125ms/step - loss: 1.7150 - accuracy: 0.4602 - val_loss: 2.3662 - val_accuracy: 0.3119
Epoch 7/100
206/206 [==============================] - 26s 125ms/step - loss: 1.4363 - accuracy: 0.5523 - val_loss: 35.5254 